In [ ]:
import pandas as pd
import matplotlib
import xlrd
from matplotlib import pyplot as plt
%matplotlib inline


In [ ]:
A = pd.read_csv("~/Documents/Insight/massdriver/data/CrashDataExport_20160603_1053.csv", sep='^', low_memory=False)

In [ ]:
A.columns

In [ ]:
fig=plt.figure(figsize=(20,10), dpi=180)
ax=fig.add_subplot(111)
plt.plot(A["X"], A["Y"], 'k+')
ax.set_aspect("equal")
ax.set_title("All reported accidents, 01/01/2013 through 01/02/2013")
ax.set_xlabel("Meters")
ax.set_ylabel("Meters")
plt.savefig("/home/louisf/Documents/Insight/massdriver/figures/massAccidents1.png", bbox_inches="tight")

In [ ]:
sum(pd.isnull(A["Road Segment ID"]))

In [ ]:
A["Crash Date"] = pd.to_datetime(A["Crash Date"])

In [ ]:
A.groupby(A["Crash Date"].dt.month).count().plot(kind="bar")

In [ ]:
import inventoryCleaner as IC
IC.insertIntoDB('roadInventory', 'segments', {"RoadInvent":668, "CRN":667, "StreetName":"TEST AVE", "SpeedLimit":65, "Terrain":1, "Structural":0})

In [ ]:
"INSERT INTO segments VALUES(999,999,'TEST ROAD', 65, 0, 3)"

In [ ]:
import accidentHandler as aH
import pandas as pd
%matplotlib inline

path = '/home/louisf/Documents/Insight/massdriver/data/crash_data/'
paths = aH.findcsvlist(path)
AT = aH.AccidentTable()

In [ ]:
AT.gatherdata(paths)

In [ ]:
AT.cleanData()

In [ ]:
AT.tablewrite('roadInventory', 'accidents', 'louisf', 'password')

In [ ]:
import csv
d = dict()
with open(l[0], 'r') as file:
    reader = csv.DictReader(file, delimiter = '^')
    for row in reader:
        d[row['Crash Number']] = row

In [ ]:
AT.accidentDF.columns

In [ ]:
AT.accidentDF['X'].plot()
AT.accidentDF['Y'].plot()
AT.accidentDF.plot(x='X', y='Y', kind='scatter')

In [ ]:
d2 = AT.accidentDF[AT.accidentDF['Is Geocoded']=='Yes']
len(d2)

In [ ]:
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
dbname = 'roadInventory'
dbuser = 'louisf'
pswd = 'password'
engine = create_engine('postgresql://%s:%s@localhost/%s'%(dbuser,pswd,dbname))
print(engine.url)

try:
    con = None
    con = psycopg2.connect(database=dbname, user=dbuser)
except psycopg2.DatabaseError as e:
    print("I cannot connect to the database " + dbname)
    print(e)
    
#AT.accidents.to_sql('accidents', engine, if_exists='append')

In [ ]:
con = psycopg2.connect(database=dbname, user=dbuser)

sql_query = """
SELECT roadsegmentid, count(*)
FROM accidents
GROUP BY roadsegmentid;
"""
accidentSegments = pd.read_sql_query(sql_query,con)

In [ ]:
sql_query="""
SELECT roadsegmen
FROM segments
"""
roadsegs = pd.read_sql_query(sql_query, con)


In [ ]:
query="""
SELECT *
FROM segments"""
segmentsTable = pd.read_sql_query(query, con)
con.close()

In [ ]:
segmentsTable.set_index('roadsegmen')

In [ ]:
accidentSegments.columns=["roadsegmen", 'count']

In [ ]:
n = pd.merge(accidentSegments, roadsegs, how='inner', on=['roadsegmen'])
n.set_index('roadsegmen')

In [ ]:
len(n)

In [ ]:
new = pd.merge(n, segmentsTable, how = 'outer', on='roadsegmen')
new.drop('naccidents',1,inplace=True)

In [ ]:
new.rename(columns={'count':'naccidents'})

In [ ]:
new

In [ ]:
segmentsTable['roadsegmen'].iloc[100]

In [ ]:
con = psycopg2.connect(database=dbname, user=dbuser)
query = """
SELECT * FROM segments WHERE roadsegmen=92336
"""
dupes = pd.read_sql_query(query, con)
con.close()

In [ ]:
sum(n['count'])

In [ ]:
con = psycopg2.connect(database=dbname, user=dbuser)
cur = con.cursor()

for index, row in n.iterrows():
    sql = 'UPDATE segments SET naccidents = %s WHERE roadsegmen = %s'
    cur.execute(sql, (row['count'], row['roadsegmen']))
con.commit()

con.close()

In [ ]:
con = psycopg2.connect(database=dbname, user=dbuser)
cur = con.cursor()

sq=''

# for index, row in n.iterrows():
#    sql = 'UPDATE segments SET naccidents = %s WHERE roadsegmen = %s'
#    cur.execute(sql, (row['count'], row['roadsegmen']))
for i in range(0,100):
    row = n.iloc[i]
    sq += 'UPDATE segments SET naccidents = {0} WHERE roadsegmen = {1};'.format(
    int(row['count']), row['roadsegmen'])
print(sq)
cur.execute(sq)
con.commit()
con.close()

In [ ]:
con = psycopg2.connect(database=dbname, user=dbuser)
query = """
SELECT naccidents FROM segments
"""
dupes = pd.read_sql_query(query, con)
con.close()

In [ ]:
sq

In [ ]:
con.close()

In [1]:
import inventoryCleaner as iC
import pandas as pd
import numpy as np
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
from sqlalchemy import types
dbname = 'roadInventory'
dbuser = 'louisf'
pswd = 'password'
engine = create_engine('postgresql://%s:%s@localhost/%s'%(dbuser,pswd,dbname))
fpath = '/home/louisf/Documents/Insight/massdriver/data/raw/shapefile/RoadInventory.geojson'
pdinv = iC.PDInventory('roadInventory', 'test')

con = psycopg2.connect(database=dbname, user=dbuser)

In [2]:
pdinv.geojsontodf(fpath)

KeyboardInterrupt: 

In [ ]:
for index, row in accidentSegments.iterrows():
    #pdinv.table[pdinv.table['roadsegmen']==row['roadsegmen']]['naccidents']=row['count']
    pdinv.table.loc[pdinv.table['roadsegmen']==row['roadsegmen'],'naccidents']=row['count']

In [ ]:
pdinv.table.to_csv('/home/louisf/Documents/Insight/massdriver/bkp.csv')

In [24]:
pdinv.table.to_sql('segments_full', engine, if_exists='replace', chunksize=20000, dtype=nametype)

In [3]:
pdinv.table = pd.DataFrame.from_csv('/home/louisf/Documents/Insight/massdriver/bkp.csv')

/home/louisf/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2825: DtypeWarning: Columns (17,27,32,60,73) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [21]:
nametype = {}
colnames = pdinv.table.columns
it = 0
for n in pdinv.table.dtypes:
    if n=='int64':
        nametype[colnames[it]]=types.INTEGER
    if n=='float64':
        nametype[colnames[it]]=types.FLOAT
    if n=='object':
        nametype[colnames[it]]=types.VARCHAR
    it +=1

In [23]:
pdinv.table.dtypes

accesscont      int64
addedroadt      int64
adt             int64
adtderivat      int64
adtstation      int64
adtyear         int64
assigned_1      int64
assignedle    float64
city            int64
county         object
crn            object
curbs           int64
dateactive     object
equation_1    float64
equationro    float64
facilityty      int64
federalaid     object
federalfun      int64
fromcity        int64
fromendtyp      int64
frommeasur    float64
fromstate       int64
fromstreet     object
functional      int64
futurefaci      int64
hpmscode        int64
hpmssample     object
iri             int64
iristatus       int64
iriyear         int64
               ...   
routenumbe     object
routequali      int64
routesyste     object
routeto       float64
routetype       int64
rpa            object
shape_len     float64
speedlimit      int64
streetlist      int64
streetna_1     object
streetname     object
streetoper      int64
structural      int64
subroute       object
surfacetyp